In [242]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import datetime

def days_between(d1, d2):
    d1 = d1.strftime("%Y-%m-%d")
    d2 = d2.strftime("%Y-%m-%d")
    return abs((d2 - d1).days)

cols1 = ['country','age','date_hospitalised','date_death','date_recovered','date_onset','date_first_consultation']
cols2 = ['sex','birth_year','country','exposure_start','confirmed_date','released_date','deceased_date']
cols3 = ['Age','Gender','Nationality','Symptomatic ToConfirmation','Days ToRecover']
cols1_rename = ['country','age','hospit_date','death_date','rec_date','onset_date','confirm_date']
cols2_rename = ['sex', 'birth_year', 'country', 'exposure_start','confirm_date','releas_date','decease_date']
cols3_rename = ['age', 'sex', 'country', 'sympt_to_confirm', 'confirm_to_release']

data1 = pd.read_csv(r"data/subset_international_cases_2020_03_11.csv", usecols=cols1)
data2 = pd.read_csv(r"data/patients.csv", usecols=cols2)
data3 = pd.read_csv(r"data/data (30).csv", usecols=cols3)

data1 = data1.rename(columns={k:v for k,v in zip(cols1,cols1_rename)})
data2 = data2.rename(columns={k:v for k,v in zip(cols2,cols2_rename)})
data3 = data3.rename(columns={k:v for k,v in zip(cols3,cols3_rename)})
cols1 = cols1_rename
cols2 = cols2_rename
cols3 = cols3_rename

# data 1
for col in cols1[2:]:
    data1[col] = pd.to_datetime(data1[col], format='%d/%m/%Y')
    
data1['onset_to_confirm'] = (data1['confirm_date']-data1['onset_date']).dt.days
data1['hospit_to_decease'] = (data1['death_date']-data1['hospit_date']).dt.days
data1['hospit_to_release'] = (data1['rec_date']-data1['hospit_date']).dt.days
data1 = data1.drop(columns=['hospit_date', 'death_date', 'rec_date', 'onset_date', 'confirm_date'])
data1.loc[data1['hospit_to_decease'].notna()]

# data 2
data2['age'] = 2020 - data2['birth_year']
for col in cols2[3:]:
    data2[col] = pd.to_datetime(data2[col])
data2['expose_to_confirm'] = (data2['confirm_date']-data2['exposure_start']).dt.days
data2['confirm_to_release'] = (data2['releas_date']-data2['confirm_date']).dt.days
data2['confirm_to_decease'] = (data2['decease_date']-data2['confirm_date']).dt.days
data2 = data2.drop(columns=['birth_year', 'exposure_start', 'confirm_date', 'confirm_date', 'releas_date', 'decease_date'])
data2 = data2.loc[data2['country'] == 'Korea']
data2 = data2.replace('Korea', 'South Korea')

# data 3
data3 = data3.loc[data3['country'] == 'Taiwanese']  # in Taiwan: confirm = hospital
data3 = data3.replace('Taiwanese', 'Taiwan')
data3['sex'] = data3['sex'].str.lower()

print(data1['country'].unique())
print(data2['country'].unique())
print(data3['country'].unique())

data1 = data1.loc[data1['onset_to_confirm'].notna()]      # 173 cases
# data1 = data1.loc[data1['hospit_to_release'].notna()]     # 51 cases
# data1 = data1.loc[data1['hospit_to_decease'].notna()]     # only 6 cases

data2 = data2.loc[data2['expose_to_confirm'].notna()]     # 352 cases
# data2 = data2.loc[data2['confirm_to_release'].notna()]    # 50 cases   # in Taiwan: confirm = hospital
# data2 = data2.loc[data2['confirm_to_decease'].notna()]    # 35 cases

data3 = data3.loc[data3['sympt_to_confirm'].notna()]      # 275 cases
# data3 = data3.loc[data3['confirm_to_release'].notna()]    # only 9 cases  # otherwise isolated

print(len(data1))
data1

['Japan' 'Singapore' 'South Korea' 'HK SAR']
['South Korea']
['Taiwan']
173


,country,age,onset_to_confirm,hospit_to_decease,hospit_to_release
11,Japan,50.0,2.0,NaN,19.0
51,Japan,60.0,0.0,NaN,NaN
58,Japan,50.0,0.0,NaN,NaN
59,Japan,50.0,1.0,NaN,NaN
62,Japan,30.0,1.0,NaN,NaN
...,...,...,...,...,...
441,HK SAR,80.0,0.0,NaN,NaN
442,HK SAR,29.0,4.0,NaN,NaN
443,HK SAR,26.0,1.0,NaN,NaN
444,HK SAR,70.0,0.0,NaN,NaN
